In [1]:
import sys
sys.path.append('/home/jovyan/work')
import xml.etree.ElementTree as ET
import xml.dom.minidom
import os
import shutil
import requests
import urllib.parse
import pandas as pd
import pymssql
from datetime import datetime
from urllib.parse import quote_plus
from sqlalchemy import create_engine
from minio import Minio
from utils.SendMinio import SendMinio
import json

In [ ]:
# Define o XML para a requisição
xml_body = """
<soapenv:Envelope xmlns:xsi="http://www.w3.org/2001/XMLSchema-instance" xmlns:xsd="http://www.w3.org/2001/XMLSchema" xmlns:soapenv="http://schemas.xmlsoap.org/soap/envelope/" xmlns:sei="Sei">
   <soapenv:Header/>
   <soapenv:Body>
      <sei:listarCidades soapenv:encodingStyle="http://schemas.xmlsoap.org/soap/encoding/">
         <SiglaSistema xsi:type="xsd:string">SIS-JFAL</SiglaSistema>
         <IdentificacaoServico xsi:type="xsd:string">{TOKEN_API}</IdentificacaoServico>
         <IdUnidade xsi:type="xsd:string">110001098</IdUnidade>
      </sei:listarCidades>
   </soapenv:Body>
</soapenv:Envelope>
"""

# Define o URL do endpoint da API SOAP
url = 'https://sei.trf5.jus.br/sei/ws/SeiWS.php'

In [3]:
# Faz a requisição SOAP
response = requests.post(url, data=xml_body, headers={'Content-Type': 'text/xml'})


In [4]:
def extract_data_from_xml(root):
    columns = []
    data = {}

    # Itera sobre os elementos XML para extrair os dados
    for item in root.findall('.//item'):
        for child in item:
            if child.tag not in columns:
                columns.append(child.tag)
                data[child.tag] = []

            data[child.tag].append(child.text)

    return columns, data

In [5]:
def xml_to_dataframe(url, xml_body):
    # Faz a requisição SOAP
    response = requests.post(url, data=xml_body, headers={'Content-Type': 'text/xml'})

    # Verifica se a requisição foi bem-sucedida
    if response.status_code == 200:
        # Parseia a resposta XML
        root = ET.fromstring(response.content)
        
        # Extrai os nomes das colunas e os dados
        columns, data = extract_data_from_xml(root)
        
        # Cria o DataFrame
        df = pd.DataFrame(data, columns=columns)
        
        return df
    
    else:
        print('Erro ao fazer a requisição:', response.status_code, response.text)
        return None

In [ ]:
df_cidades = xml_to_dataframe(url, xml_body)

if df_cidades is not None:
    print(df_cidades)

# Função para inserir no Data Lake

In [7]:
def insert_datalake(df, name_folder, name_file):
    today = datetime.today().strftime("%Y%m%d")
    # Camada (bucket) do MinIO para qual os dados extraídos serão enviados
    bucket_path = "raw"
    # Data Mart do DW ao qual esse notebook pertence
    business_area = "sei_cidades"

    minio_host = os.getenv("MINIO_HOST")
    minio_access_key = os.getenv("MINIO_ACCESS_KEY")
    minio_secret_key = os.getenv("MINIO_SECRET_KEY")

    client = Minio(
        minio_host,
        access_key = minio_access_key,
        secret_key = minio_secret_key,
        secure = False
    )

    today = datetime.today().strftime("%Y%m%d")
    today_folder = datetime.today().strftime("%Y%m%d")
    file_output = f'{name_file}_{today}.json'

    # Transforme o DataFrame em uma lista de dicionários
    data = df.to_dict(orient='records')

    # Crie um arquivo JSON com o formato desejado
    with open(file_output, 'w', encoding='utf-8') as json_file:
        json.dump(data, json_file, ensure_ascii=False, indent=4)
    
    sm = SendMinio(client=client,bucket_name=bucket_path,object_name=f"{business_area}/operacional/{name_folder}/{today_folder}/{file_output}",file_path=file_output)
    sm.send2bucket()
    os.remove(file_output)

# Carga

In [8]:
name_folder = "cidades"
name_file = "cidades"

In [9]:
insert_datalake(df_cidades, name_folder, name_file)

Bucket existente... enviando arquivos
